## Evaluation

In this notebook, a small evaluation of the CNN classifier trained in the *Model_Definition.ipynb* notebook is performed. The focus is on the generation of a small data set for each font, followed by the classification of the rendered images.

In [25]:
#!pip install opencv-python

In [27]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow import keras
import seaborn as sns
import json
from sklearn.metrics import classification_report, confusion_matrix
import random
from PIL import Image, ImageDraw, ImageFont

In [28]:
def show_image(image):
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()

### Generation of Perspective Transformation (to simulate Screenshots)

In [29]:
def create_image_with_text(
    text='A',
    font_path="", 
    width=40, 
    height=40, 
    thickness=1, 
    zoom_factor=1.0  
):
    try:
        image = np.ones((height, width), dtype=np.uint8) * 255 
        font_size = 30  # Set Font Size
        font = ImageFont.truetype(font_path, font_size)
        
        pil_image = Image.fromarray(image)
        draw = ImageDraw.Draw(pil_image)
        
        text_width = font.getlength(text)
        text_height = font_size
        
        # Center Image
        text_x = (width - text_width) // 2 
        text_y = (height - text_height) // 2
        
        draw.text((text_x, text_y), text, font=font, fill=0)
        
        image = np.array(pil_image)

        pts1 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
        
        zoomed_width = int(width * zoom_factor)
        zoomed_height = int(height * zoom_factor)
        
        pts2 = np.float32([
            [random.randint(-zoomed_width // 4, zoomed_width // 4), random.randint(-zoomed_height // 4, zoomed_height // 4)],
            [zoomed_width - random.randint(-zoomed_width // 4, zoomed_width // 4), random.randint(-zoomed_height // 4, zoomed_height // 4)],
            [random.randint(-zoomed_width // 4, zoomed_width // 4), zoomed_height - random.randint(-zoomed_height // 4, zoomed_height // 4)],
            [zoomed_width - random.randint(-zoomed_width // 4, zoomed_width // 4), zoomed_height - random.randint(-zoomed_height // 4, zoomed_height // 4)]
        ])
        
        M = cv2.getPerspectiveTransform(pts1, pts2)
        image = cv2.warpPerspective(image, M, (zoomed_width, zoomed_height))
        
        return image
    except Exception as e:
        print(f"Fehler beim Öffnen der Datei: {font_path}")
        print(f"Absoluter Pfad: {os.path.abspath(font_path)}")
        print(f"Fehlermeldung: {str(e)}")
        return None


In [30]:
serif_folder = "Font/Serif"
no_serif_folder = "Font/No_Serif"

def receive_dataset(serif = False):
    font_images = {}
    folder = serif_folder if serif else no_serif_folder
    for font_file in os.listdir(folder):
        if font_file.endswith(('.ttf', '.otf')):  
            font_path = os.path.join(folder, font_file)
            font_name = os.path.splitext(font_file)[0]  

            img_list = []

            for i in range(26):
                letter = chr(ord('a') + i)
                image = create_image_with_text(text=letter, font_path=font_path)
                img_list.append(image)

            font_images[font_name] = img_list
    return font_images

In [31]:
data_serif = receive_dataset(True)

In [32]:
data_no_serif = receive_dataset(False)

### Load Model

In [33]:
model = keras.models.load_model("Model/classification.keras")

with open('Model/font_map.json', 'r') as file:
    data_dict = json.load(file)

fontmap = data_dict

In [34]:
def predict_font(image, model, font_map):
    if len(image.shape) == 2:
        img = image
    elif len(image.shape) == 3 and image.shape[2] == 1:
        img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        img = image
    
    img = cv2.resize(img, (28, 28))
   # show_image(img)
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    inverse_font_map = {v: k for k, v in font_map.items()}
    predicted_font = inverse_font_map[predicted_class]
    confidence = np.max(prediction) * 100
    return predicted_font, confidence

In [35]:
def calculate_predictions(dictionary):
    total_predictions = 0
    total_correct = 0
    font_accuracies = {}

    for font, images in dictionary.items():
        font_predictions = 0
        font_correct = 0

        for image in images:
            try:
                predicted_font, confidence = predict_font(image, model, fontmap)
                font_predictions += 1
                total_predictions += 1

                if predicted_font == font:
                    font_correct += 1
                    total_correct += 1
            except Exception as e:
                print(f"Error during prediction for {font}: {str(e)}")

        if font_predictions > 0:
            font_accuracy = (font_correct / font_predictions) * 100
            font_accuracies[font] = font_accuracy
            print(f"{font}: {font_accuracy:.2f}% ({font_correct}/{font_predictions})")

    overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
    print(f"\nClassification complete.")
    print(f"Overall Accuracy: {overall_accuracy:.2f}% ({total_correct}/{total_predictions})")

    if font_accuracies:
        best_font = max(font_accuracies, key=font_accuracies.get)
        worst_font = min(font_accuracies, key=font_accuracies.get)
        print(f"\nBest performing font: {best_font} ({font_accuracies[best_font]:.2f}%)")
        print(f"Worst performing font: {worst_font} ({font_accuracies[worst_font]:.2f}%)")

In [36]:
calculate_predictions(data_serif)

1/1 [==============================] - 0s 16ms/step
courier: 96.15% (25/26)
1/1 [==============================] - 0s 16ms/step
palatino: 23.08% (6/26)
1/1 [==============================] - 0s 29ms/step
georgia: 61.54% (16/26)
1/1 [==============================] - 0s 16ms/step
Times New Roman: 26.92% (7/26)
1/1 [==============================] - 0s 16ms/step
Rockwell: 65.38% (17/26)
1/1 [==============================] - 0s 15ms/step
garamond: 80.77% (21/26)
1/1 [==============================] - 0s 16ms/step
bookantiqua: 42.31% (11/26)
1/1 [==============================] - 0s 16ms/step
baskerville: 46.15% (12/26)
1/1 [==============================] - 0s 15ms/step
Didot: 96.15% (25/26)
1/1 [==============================] - 0s 16ms/step
Merriweather: 88.46% (23/26)

Classification complete.
Overall Accuracy: 62.69% (163/260)

Best performing font: courier (96.15%)
Worst performing font: palatino (23.08%)


In [37]:
calculate_predictions(data_no_serif)

1/1 [==============================] - 0s 16ms/step
Roboto-Black: 96.15% (25/26)
1/1 [==============================] - 0s 19ms/step
CenturyGothicPaneuropeanBlack: 100.00% (26/26)
1/1 [==============================] - 0s 16ms/step
GillSansStd: 73.08% (19/26)
1/1 [==============================] - 0s 16ms/step
OpenSans-Light: 23.08% (6/26)
1/1 [==============================] - 0s 16ms/step
Montserrat-Light: 88.46% (23/26)
1/1 [==============================] - 0s 16ms/step
Helvetica: 84.62% (22/26)
1/1 [==============================] - 0s 18ms/step
ARIAL: 38.46% (10/26)
1/1 [==============================] - 0s 15ms/step
calibril: 96.15% (25/26)
1/1 [==============================] - 0s 16ms/step
Verdana: 84.62% (22/26)
1/1 [==============================] - 0s 15ms/step
FuturaCyrillicMedium: 57.69% (15/26)

Classification complete.
Overall Accuracy: 74.23% (193/260)

Best performing font: CenturyGothicPaneuropeanBlack (100.00%)
Worst performing font: OpenSans-Light (23.08%)
